# Métodos de iteración funcional para sistemas de ecuaciones

## Declaración de métodos y funciones auxiliares

Entradas:

 - una función $g: \mathbb{R}^N \to \mathbb{R}^N, g\in \mathscr{C}^1(\mathbb{R}^N)$,
 - $x_0 \in \mathbb{R}^N$.
 
y, según la variante:
 
  1. tolerancia entre dos iteraciones consecutivas,
  2. solución y tolerancia,
  3. constante de Lipstchiz y tolerancia.

Primero, definimos un generador de los términos de la sucesión para cada método.

In [1]:
def functional_iteration(g, x_0):
    
    x = vector(x_0)
    while True:
        yield x
        x = g(*x).n()

En el método de Newton, resolvemos el SEL correspondiente en lugar de hallar la inversa de la matriz.

In [2]:
def newton(g, x_0):
    x = vector(x_0)
    f = g.derivative()
    while True:
        yield x
        y = f(*x).solve_right(g(*x))
        x = vector(x) - vector(y)

Para la primera variante, creamos un generador que, a partir de uno de los métodos, nos dé la distancia entre dos aproximaciones consecutivas, y nos quedamos con el primer elemento del generador para el que la distancia con la aproximación anterior sea menor que la tolerancia dada.

In [3]:
def adjacent_distance(method, g, x_0):
    gen = method(g, x_0)
    x = gen.next()
    
    for i in gen:
        yield (i,(x-i).norm(oo))
        x = i

In [4]:
import itertools as it

def first_variant(method, g, x_0, epsilon):
    orig_gen = adjacent_distance(method,g,x_0)
    gen = it.takewhile(lambda (x,e) : e >= epsilon, orig_gen)
    
    return it.chain(gen, it.islice(orig_gen,1))

Para la segunda variante, procedemos de forma similar y creamos un generador que pare en la primera aproximación que nos proporcione un error menor que la tolerancia.

In [5]:
def second_variant(method, g, x_0, sol, epsilon):
    orig_gen = method(g,x_0)
    gen = it.takewhile(lambda (x) : (x-sol).norm() >= epsilon, orig_gen)
    
    return it.chain(gen, it.islice(orig_gen,2))

En este caso, tenemos $L\in (L_0,1)$, con $L_0$ la constante de Lipschitz de $g$, y el teorema del punto fijo nos da la cota

$$|x_n-x^*| \le \frac{L^n}{1-L}|g(x_0)-x_0|$$

Ahora, si imponemos 

$$\frac{L^n}{1-L}|g(x_0)-x_0| < \varepsilon$$

tenemos que

$$ n > \frac{\log\left(\displaystyle\frac{\varepsilon(1-L)}{|g(x_0)-x_0|}\right)}{\log(L)} $$

In [6]:
def third_variant(g, x_0, L, epsilon):
    n_iterations = int((log(epsilon*(1-L)/(vector(g(*x_0))-vector(x_0)).norm())/log(L)))+2
    
    return it.islice(functional_iteration(g,x_0), n_iterations)

In [7]:
def print_all(gen):
    for i in gen:
        print(i)
        
def print_last(gen):
    j = None
    for j in gen:
        pass
    print(j)

# Ejemplos

Tomamos la función:
$$
    f(x,y) = ( y - \arctan(x), x - \arctan(y))\\
$$

Así, podemos definir dos formas de aplicar la iteración funcional.
La primera sería $g_1$:

$$
    \begin{cases}
    y=\arctan(x) \\
    x = \arctan(x)
    \end{cases}
$$

Y la segunda sería $g_2$:
$$
    \begin{cases}
    y=\tan(x) \\
    x = \tan(y)
    \end{cases}
$$

La tangente es claramente no contractiva, por lo que escogeremos ahora el dominio para la función $g_1$ que será: $D= \left(-\displaystyle\frac{\pi}{2}, \displaystyle\frac{\pi}{2}\right)$. En este dominio, $g_1(D) \subseteq D$.

Ahora, analizamos lo que se ha pedido:

In [28]:
import sys

f(x,y) = (y-arctan(x), x-arctan(y))

g_1(x,y) = (arctan(y), arctan(x))
g_2(x,y) = (tan(x), tan(y))

In [29]:
print("Todos las aproximaciones del primer método, primera variante")
gen = first_variant(functional_iteration, g_1, (0.5,.5), 1e-3)

for fix_point, error in gen:
    print("Aproximación: {}, distancia entre términos: {}".format(fix_point, error))

Todos las aproximaciones del primer método, primera variante
Aproximación: (0.463647609000806, 0.463647609000806), distancia entre términos: 0.0363523909991939
Aproximación: (0.434145140168418, 0.434145140168418), distancia entre términos: 0.0295024688323881
Aproximación: (0.409591093415996, 0.409591093415996), distancia entre términos: 0.0245540467524215
Aproximación: (0.388747119537355, 0.388747119537355), distancia entre términos: 0.0208439738786419
Aproximación: (0.370768137419872, 0.370768137419872), distancia entre términos: 0.0179789821174821
Aproximación: (0.355055392187276, 0.355055392187276), distancia entre términos: 0.0157127452325966
Aproximación: (0.341171391269379, 0.341171391269379), distancia entre términos: 0.0138840009178970
Aproximación: (0.328788141773278, 0.328788141773278), distancia entre términos: 0.0123832494961010
Aproximación: (0.317654319420148, 0.317654319420148), distancia entre términos: 0.0111338223531300
Aproximación: (0.307573703539942, 0.307573703539

In [30]:
print("Aproximacion de la solución por el metodo de Newton.")
gen = first_variant(newton, f, (0.5,0.5), 1e-3)
    
for fix_point, error in gen:
    print("Aproximación: {}, error: {}".format(fix_point, error))

Aproximacion de la solución por el metodo de Newton.
Aproximación: (0.318238045004030, 0.318238045004030), error: 0.181761954995970
Aproximación: (0.208038059631220, 0.208038059631220), error: 0.110199985372810
Aproximación: (0.137513272689158, 0.137513272689158), error: 0.0705247869420625
Aproximación: (0.0913315808523198, 0.0913315808523198), error: 0.0461816918368380
Aproximación: (0.0607865035595681, 0.0607865035595681), error: 0.0305450772927517
Aproximación: (0.0404944355571086, 0.0404944355571086), error: 0.0202920680024595
Aproximación: (0.0269874428881970, 0.0269874428881970), error: 0.0135069926689116
Aproximación: (0.0179890086697816, 0.0179890086697816), error: 0.00899843421841546
Aproximación: (0.0119918963777518, 0.0119918963777518), error: 0.00599711229202976
Aproximación: (0.00799436766579522, 0.00799436766579522), error: 0.00399752871195660
Aproximación: (0.00532951032313835, 0.00532951032313835), error: 0.00266485734265687
Aproximación: (0.00355298669865907, 0.0035529

In [31]:
print("Algunas iteraciones del segundo método")
gen = first_variant(functional_iteration, g_2, (1,1), 1e-5)

for fix_point, error in it.islice(gen,1,10):
    print("Aproximación: {}, distancia entre término: {}".format(fix_point, error))

Algunas iteraciones del segundo método
Aproximación: (74.6859333987654, 74.6859333987654), distancia entre término: 73.1285256741105
Aproximación: (-0.863518854877450, -0.863518854877450), distancia entre término: 75.5494522536428
Aproximación: (-1.16985635505842, -1.16985635505842), distancia entre término: 0.306337500180974
Aproximación: (-2.35903773417943, -2.35903773417943), distancia entre término: 1.18918137912101
Aproximación: (0.994329619022585, 0.994329619022585), distancia entre término: 3.35336735320201
Aproximación: (1.53815355692099, 1.53815355692099), distancia entre término: 0.543823937898405
Aproximación: (30.6237735079718, 30.6237735079718), distancia entre término: 29.0856199510508
Aproximación: (-1.01360181434666, -1.01360181434666), distancia entre término: 31.6373753223185
Aproximación: (-1.60501236782676, -1.60501236782676), distancia entre término: 0.591410553480097


In [25]:
print("Última aproximación del primer método, segunda variante")
gen = second_variant(functional_iteration, g_1, (0.5,.5), vector([0, 0]), 1e-2)

for num, fix_point in enumerate(gen):
    pass
print("Aproximación {}: {}, error: {}".format(num, fix_point, fix_point.norm()))


Última aproximación del primer método, segunda variante
Aproximación 29997: (0.00707074666520548, 0.00707074666520548), error: 0.00999954583003793


La convergencia de este método es muy lenta, y no conocemos la constante de Lipschitz.

In [27]:
print("Última aproximación del primer método, tercera variante")
gen = third_variant(g_1, (0.5,.5), 0.9999, 1e-2)

for num, fix_point in enumerate(gen):
    pass
print("Aproximación {}: {}, error: {}".format(num, fix_point, fix_point.norm()))

Última aproximación del primer método, tercera variante
Aproximación 108471: (0.00371860230638647, 0.00371860230638647), error: 0.00525889781476362


In [28]:
print("Aproximación de la solución por el método de Newton, segunda variante.")
gen = second_variant(newton, f, (0.5,0.5), vector([0,0]), 1e-2)

for num, fix_point in enumerate(gen):
    print("Aproximacion {}: {}, error: {}.".format(num, fix_point, fix_point.norm()))

Aproximación de la solución por el método de Newton, segunda variante.
Aproximacion 0: (0.500000000000000, 0.500000000000000), error: 0.707106781186548.
Aproximacion 1: (0.318238045004030, 0.318238045004030), error: 0.450056559307799.
Aproximacion 2: (0.208038059631220, 0.208038059631220), error: 0.294210245420254.
Aproximacion 3: (0.137513272689158, 0.137513272689158), error: 0.194473135243317.
Aproximacion 4: (0.0913315808523198, 0.0913315808523198), error: 0.129162360314326.
Aproximacion 5: (0.0607865035595681, 0.0607865035595681), error: 0.0859650977431816.
Aproximacion 6: (0.0404944355571086, 0.0404944355571086), error: 0.0572677799655063.
Aproximacion 7: (0.0269874428881970, 0.0269874428881970), error: 0.0381660077462576.
Aproximacion 8: (0.0179890086697816, 0.0179890086697816), error: 0.0254403000344523.
Aproximacion 9: (0.0119918963777518, 0.0119918963777518), error: 0.0169591024959894.
Aproximacion 10: (0.00799436766579522, 0.00799436766579522), error: 0.0113057431755645.
Apro

# Ejercicios

Empecemos con el primer apartado del ejercicio 21 en el que compararemos las ejecuciones y resultados del ejercicio 18, en este se nos pedía resolverlo con el método de convergencia funcional. Ahora nos piden resolverlo con el método de Newton y que comparemos la eficiencia y resultados de ambos métodos.

## Ejercicio 18

In [8]:
ejercicio18Iter(x,y,z)=((cos(y*z)+1/2)/3, sqrt((sin(z)+x^2+1.06)/81)-0.1, -(exp(-x*y)+(10*pi.n()-3)/3)/20)
ejercicio18New(x,y,z)=(-cos(y*z)+3*x-0.5, sin(z)+x^2-81*(y+0.1)^2+1.06 , e^(-x*y)+20*z+(10*pi.n()-3)/3)
print("Todos las aproximaciones de la primera variante con el método de Newton")
Primera = first_variant(newton, ejercicio18New, (0.1,0.1,-0.1), 10^(-5))

fix_point, error = None, None
for indice,(fix_point,error) in enumerate(Primera):
    print("{}: \n\tAproximación: {}, \n\tpunto: {} \n\tdistancia entre términos: {}".format(indice+1,fix_point, ejercicio18New(*fix_point), error))
    

    
print("\n\nTodos las aproximaciones de la primera variante con el método de Iteración Funcional")
Primera = first_variant(functional_iteration, ejercicio18Iter, (0.1,0.1,-0.1), 10^(-5))

fix_point, error = None, None
for indice,(fix_point,error) in enumerate(Primera):
    print("{}: \n\tAproximación: {}, \n\tpunto: {} \n\tdistancia entre términos: {}\n".format(indice+1, fix_point, ejercicio18Iter(*fix_point), error))
    

Todos las aproximaciones de la primera variante con el método de Newton
1: 
	Aproximación: (0.499869672926428, 0.0194668485374181, -0.521520471935831), 
	punto: (-0.000339446456205650, -0.344387927635969, 0.0318823779239974) 
	distancia entre términos: 0.421520471935831
2: 
	Aproximación: (0.500014240164219, 0.00158859137029389, -0.523556964347638), 
	punto: (0.0000430663701153078, -0.0258891432153728, 0.0000422220935369921) 
	distancia entre términos: 0.0178782571671242
3: 
	Aproximación: (0.500000113467834, 0.0000124447833215471, -0.523598450072889), 
	punto: (3.40424732292455e-7, -0.000201222653344435, 2.88134476278401e-7) 
	distancia entre términos: 0.00157614658697235
4: 
	Aproximación: (0.500000000007076, 7.75785723143997e-10, -0.523598775578007), 
	punto: (2.12270201416231e-11, -1.25430800190784e-8, 1.79447567916213e-11) 
	distancia entre términos: 0.0000124440075358239
5: 
	Aproximación: (0.500000000000000, 8.32164840773978e-18, -0.523598775598299), 
	punto: (0.000000000000000,

Como podemos ver, con el método de Newton hemos necesitado una iteración menos para alcanzar la tolerancia deseada dentro del contexto de la distancia entre dos términos. No sólo eso, sino que, con una iteración menos hemos conseguido que la distancia entre términos sea 10^8 veces menor. 

Por ello podemos asegurar que, en este caso particular, el método de Newton converge más rápidamente.

A continuación haremos la misma prueba, pero esta vez con el ejercicio 20.

## Ejercicio 20

In [12]:
ejercicio20Iter(x,y)=(y/sqrt(5),(sin(x)+cos(y))/4)
ejercicio20New(x,y)=(5*x^2-y^2,y-(sin(x)+cos(y))/4)

print("\n\nTodos las aproximaciones de la primera variante con el método de Iteración Funcional")
Segunda = first_variant(functional_iteration, ejercicio20Iter, (1/4,1/4), 10^(-5))

fix_point, error = None, None
for indice,(fix_point,error) in enumerate(Segunda):
    print("{}: \n\tAproximación: {}, \n\tpunto: {} \n\tdistancia entre términos: {}\n".format(indice+1, fix_point, ejercicio20Iter(*fix_point), error))




Todos las aproximaciones de la primera variante con el método de Iteración Funcional
1: 
	Aproximación: (0.111803398874989, 0.304079095241292), 
	punto: (0.0608158190482584*sqrt(5), 0.266423427535425) 
	distancia entre términos: 0.138196601125011

2: 
	Aproximación: (0.135988305499232, 0.266423427535425), 
	punto: (0.0532846855070850*sqrt(5), 0.275072068196639) 
	distancia entre términos: 0.0376556677058668

3: 
	Aproximación: (0.119148178953540, 0.275072068196639), 
	punto: (0.0550144136393278*sqrt(5), 0.270318023474851) 
	distancia entre términos: 0.0168401265456923

4: 
	Aproximación: (0.123015968639829, 0.270318023474851), 
	punto: (0.0540636046949702*sqrt(5), 0.271597989694383) 
	distancia entre términos: 0.00475404472178792

5: 
	Aproximación: (0.120889895206630, 0.271597989694383), 
	punto: (0.0543195979388767*sqrt(5), 0.270984771837357) 
	distancia entre términos: 0.00212607343319837

6: 
	Aproximación: (0.121462313501786, 0.270984771837357), 
	punto: (0.0541969543674714*sqrt

In [ ]:
print("Todos las aproximaciones de la primera variante con el método de Newton")
Segunda = first_variant(newton, ejercicio20New, (1/4,1/4), 10^(-5))

fix_point, error = None, None
for indice,(fix_point,error) in enumerate(Segunda):
    print("{}: \n\tAproximación: {}, \n\tpunto: {} \n\tdistancia entre términos: {}".format(indice+1,fix_point, ejercicio20New(*fix_point), error))
    


Como podemos ver, en este caso el coste computacional es demasiado grande en el caso del método de Newton. Sin embargo, en el método de iteración funcional el coste computacional es mínimo en comparación.

Tras hacer ambas comparaciones con ambos ejercicios, vemos que, conforme están actualmente diseñados estos algoritmos, el algoritmo de iteración funcional es mucho más eficiente.